In [ ]:
from __future__ import print_function
from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
from collections import defaultdict

from util_gradient import get_mse, pack, unpack
from util_gradient import objective, gradient_user, gradient_item, gradient
from util_gradient import predict_one_rating

In [ ]:
def init_theta(K, num_users, num_items):
    alpha, beta_users, beta_items = pickle.load(open("alpha_beta_init.feature", "rb"))
    # alpha = np.mean(train_rating_array[:, 2])
    # beta_users = np.random.normal(0, 0.5, (num_users, ))
    # beta_items = np.random.normal(0, 0.5, (num_items, ))
    # gamma_users = np.random.normal(0, 0.5, (num_users, K))
    # gamma_items = np.random.normal(0, 0.5, (num_items, K))
    gamma_users = np.zeros((num_users, K))
    gamma_items = np.zeros((num_items, K))
    
    theta_length = (1 + 
                    beta_users.size + 
                    beta_items.size +
                    gamma_users.size + 
                    gamma_items.size)
    
    theta = np.empty((theta_length, ))
    return pack(theta, K, alpha, beta_users, beta_items, gamma_users, gamma_items)

In [ ]:
def progress_callback(theta):
    def test_and_get_mse(rating_array, theta, K):
        ratings_predict = [predict_one_rating(user_index, item_index, theta, K) 
                           for user_index, item_index in rating_array[:, :2]]
        ratings = rating_array[:, 2]
        return get_mse(ratings_predict, ratings)
    
    print("train mse:", test_and_get_mse(train_rating_array, theta, K))
    print("valid mse:", test_and_get_mse(valid_rating_array, theta, K))

In [ ]:
from scipy.optimize import minimize, fmin_l_bfgs_b

# load preprocessed data
# id <-> index infastructure
user_index_map_id, item_index_map_id = \
    pickle.load(open("id_index_map.feature", "rb"))
# train_rating_array, valid_rating_array
train_rating_array, valid_rating_array, all_rating_array = \
    pickle.load(open("train_valid_all_rating_array.feature", "rb"))

In [ ]:
# global variableds
K = 2
lam = 1.0
num_users = len(user_index_map_id)
num_items = len(item_index_map_id)
assert num_users == 35736
assert num_items == 37801

# init theta and grad_buffer
theta = init_theta(K, num_users, num_items)
grad_buffer = np.zeros_like(theta)

# print init
progress_callback(theta)

# res = fmin_l_bfgs_b(objective, theta, 
#                     fprime   = gradient,
#                     args     = (grad_buffer, train_rating_array, lam, K),
#                     maxiter  = 10,
#                     disp     = 1,
#                     callback = progress_callback)
# theta = res.x

# res = minimize(objective, theta, 
#                args       = (grad_buffer, train_rating_array, lam, K),
#                method     = 'L-BFGS-B',
#                jac        = gradient, # jac=gradient_item
#                options    = {'disp': True, 'maxiter': 30},
#                callback   = progress_callback)
# theta = res.x

In [ ]:
progress_callback(theta)

In [ ]:
# # get header_str and user_item_ids to predict
# with open('pairs_Rating.txt') as f:
#     # read and strip lines
#     lines = [l.strip() for l in f.readlines()]
#     # stirip out the headers
#     header_str = lines.pop(0)
#     # get a list of user_item_ids
#     user_item_ids = [l.split('-') for l in lines]
    
# # write to output file
# f = open('predictions_Rating.txt', 'w')
# print(header_str, file=f)
# for user_id, item_id in user_item_ids:
#     rating = predict_one_rating(user_id_map_index[user_id], 
#                                 item_id_map_index[item_id], 
#                                 theta, K)
#     rating = min(5.0, rating)
#     rating = max(0.0, rating)
#     print('%s-%s,%s' % (user_id, item_id, rating), file=f)
# f.close()